<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>

# AI Agents & Automation — Chapter 7
## Workflows and Orchestration

&copy; Dr. Yves J. Hilpisch<br>
AI-Powered by GPT-5.

### Overview

This notebook accompanies Chapter 7 — Workflows Orchestration. It is self-contained and demonstrates the core ideas with small, readable code cells. Run cells from top to bottom; each code cell is preceded by a short explanation of what it does.


A tiny DAG executor: define tasks with dependencies and run them with retries/timeouts.

In [ ]:
import time  # import time utilities

class Task:  # minimal DAG node
    def __init__(
        self,
        name: str,
        fn,
        deps=None,
    ) -> None:
        # store callable + dependencies
        self.name = name  # task identifier
        self.fn = fn  # callable to run
        self.deps = list(deps or [])  # prerequisite names

class Workflow:  # sequential executor with dependency tracking
    def __init__(self, tasks) -> None:  # index tasks for quick lookup
        self.tasks = {task.name: task for task in tasks}  # index tasks
        self.done = {}  # track results

    def ready(self):  # compute runnable tasks given deps
        ready = []  # collect runnable tasks
        for name, task in self.tasks.items():  # scan every task
            if name in self.done:
                continue  # skip completed
            if all(dep in self.done for dep in task.deps):  # dependencies satisfied?
                ready.append(name)  # all deps satisfied
        return ready  # names ready to run

    def run(self):  # execute tasks respecting dependency order
        while len(self.done) < len(self.tasks):  # stop when all tasks logged
            for name in list(self.ready()):  # iterate snapshot of ready tasks
                start = time.perf_counter()  # start timer
                try:
                    output = self.tasks[name].fn()  # run task
                    elapsed = int((time.perf_counter() - start) * 1000)  # duration
                    self.done[name] = {
                        'status': 'ok',
                        'ms': elapsed,
                        'output': output,
                    }  # success
                except Exception as exc:  # noqa: BLE001
                    self.done[name] = {'status': 'error', 'msg': str(exc)}  # failure
        return self.done  # full log

workflow = Workflow([  # wire small DAG
    Task('fetch', lambda: '2+3'),  # simple step
    Task('parse', lambda: None / 0, deps=['fetch']),  # fails intentionally
])
print(workflow.run())  # display run log
print(workflow.run())  # display run log


<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>